<a href="https://colab.research.google.com/github/vtczk/imdb-bert-ktrain/blob/master/BertSentimentAnalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment analisis task is a binary classification (or sometimes multi class clasiffication if the neutral label is considered) in which the input text is assigned with the label assigning the sentece a positive or negative sentiment.In this notebook I will approach this topic using the Fast AI library ktrain. The training dataset will be standard IMDB review dataset containing positive/negative movie reviews (for more details refer to http://ai.stanford.edu/~amaas/data/sentiment/)

The network architecture will use transfer learning alongside the google BERT pretrained model to achieve very high accuracy in a single epoch of training.



In [0]:
!mkdir data

In [10]:
cd data

/content/data/data


In [11]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-01-03 22:19:52--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  74.8MB/s    in 1.1s    

2020-01-03 22:19:53 (74.8 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
 %%capture
! tar -xvf aclImdb_v1.tar.gz

In [13]:
!pip3 install ktrain

Processing /root/.cache/pip/wheels/86/20/70/000142264c6a768a4e8b67d8ef2ebc32c018684acb2eed7586/ktrain-0.7.3-cp36-none-any.whl
Processing /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68/seqeval-0.0.12-cp36-none-any.whl
Processing /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe/langdetect-1.0.7-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/4e/847feebfc3e71c773b23ee06c74687b8c50a5a6d6aaff452a0a4f4eb9a32/cchardet-2.1.5-cp36-cp36m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0/keras_bert-0.80.0-cp36-none-any.whl
Processing /root/.cache/pip/wheels/de/63/64/3699be2a9d0ccdb37c7f16329acf3863fd76eda58c39c737af/networkx-2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0a/57/496b1eab888171b0801a0a44d3245a7874b8d1cc04c1fbfdbb5e3327fc7a/keras-transformer-0.31.0.tar.gz
  Using cached https://files.pyt

In [14]:
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


In [15]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder('/content/data/aclImdb/', 
                                                                       maxlen=500, 
                                                                       preprocess_mode='bert',
                                                                       train_test_names=['train', 
                                                                                         'test'],
                                                                       classes=['pos', 'neg'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [16]:
model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [0]:
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=6)

In [0]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...
Train on 25000 samples
Epoch 1/1024
 4680/25000 [====>.........................] - ETA: 26:24 - loss: 0.5709 - acc: 0.6923

In [0]:
learner.lr_plot()

In [0]:
#Google recommended learning rate
learner.fit_onecycle(2e-5, 1)

In [0]:
learner.save_model('/tmp/mymodel')

In [0]:
learner.validate(val_data=(x_test, y_test))

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)


In [0]:
examples = ["This movie was horrible","I really enjoyed the action","This was so boring","This was rather far from great"]

predictor.predict(examples)


In [0]:

actualEndgame = ["The writers got carried away, the directors over reached and the studio sacrificed the integrity of their product."]
predictor.predict(actualEndgame)